** Método Naive Bayes para classificação de spam usando a lista Tabu (Traduzido para o português)**
====================================================
Implementado por Clyde Wang, 17 de fevereiro de 2017

Esse classificador de spam é implementado pelo Naive Bayes Model, uma solução simples, mas muito eficiente no problema de classificação de spam. Em resumo, o Naive Bayes trata todos os recursos independentemente um do outro, tornando a inferência muito eficiente. Você pode consultar [Naive Bayes] [1] para mais detalhes. Este artigo apresenta brevemente o processo de seleção da lista de tabus e criação do algoritmo de aprendizado. Espero que você goste.

Esse código funciona muito bem com a precisão de 98,31% na amostra de treinamento e 97,81% na amostra de teste.
Espero que alguém possa melhorá-lo e melhorar seu desempenho. E aqui vamos nós!

   [1]: https://en.wikipedia.org/wiki/Naive_Bayes_classifier

## 1. Configuração do ambiente ##
Antes de começarmos, precisamos configurar o ambiente, verifique se esses pacotes estão instalados no seu computador quando você copia esse código no arquivo local.

In [ ]:
#coding:utf-8
import pandas as pd
import numpy as np
import re
from sklearn.naive_bayes import BernoulliNB

2. Leia o arquivo de dados
-----------------
A primeira tarefa é ler dados do arquivo .csv, e aqui podemos tirar proveito do `pandas.DataFrame` para armazenar e processar os dados brutos. (veja [aqui] [1]) Observe que precisamos dividir nossos dados brutos em duas partes para testar a capacidade de generalização do nosso modelo. Aqui está o código:


   [1]: http://pandas.pydata.org/pandas-docs/stable/10min.html

In [ ]:
def readData():
	SMS_df = pd.read_csv('../input/spam.csv',usecols=[0,1],encoding='latin-1')
	SMS_df.columns=['label','content']
	n = int(SMS_df.shape[0])
    # split into training data and test data
	return SMS_df.iloc[:int(n/2)], SMS_df.iloc[int(n/2):]

3. Gere uma lista de tabu
--------------------
A ** tabu list ** é uma lista desses indicadores significativos de spam por SMS. Aqui, selecionamos TF-IDF como o princípio da geração de lista.

Frequência de termos (TF) é a frequência de uma palavra em um determinado tipo de documento. Se houver um artigo de 50 palavras com 2 'dados', o TF dos 'dados' será dado por '2/50 = 0,01'.

No entanto, existem algumas palavras de alta frequência em inglês, como 'a', 'is', 'are' etc. Temos que remover essas palavras da nossa lista. E aqui vem o IDF.

A Frequência inversa de documentos (IDF) é o indicador para refletir a importância de uma palavra relacionada a algum tópico específico. É dado por `log (#total de artigos / # artigos contendo w)`, por exemplo, se tivermos 5 artigos, apenas um terá a palavra 'gene' com frequência de termo de 0,002, mas todos os cinco artigos contêm a palavra 'tecnologia' da frequência do termo de 0,5, o IDF do 'gene' é `log (5/1)> 0`, mas o IDF da 'tecnologia' é` log (5/5) = 0`.

TF-IDF é o produto de TF e IDF: no exemplo acima, `TFIDF ('gene') = 0,002 * log (5/1)> 0` enquanto` TFIDF ('tecnologia') = 0,5 * log (5 / 5) = 0`, portanto, neste caso, 'gene' é um indicador melhor que 'tecnologia'.

No meu código, eu calculo o TF-IDF de cada palavra para 'spam' e 'ham' e calculo a diferença entre elas; assim, posso selecionar as palavras que são as mais representativas para a classe 'spam'.

Aqui está o código:

In [ ]:
def generate_tabu_list(path, tabu_size=200,ignore=3):
	train_df,_ = readData()
	spam_TF_dict = dict()
	ham_TF_dict = dict()
	IDF_dict = dict()

	# ignore all other than letters.
	for i in range(train_df.shape[0]):
		finds = re.findall('[A-Za-z]+', train_df.iloc[i].content)
		if train_df.iloc[i].label == 'spam':
			for find in finds:
				if len(find)<ignore: continue
				find = find.lower()
				try:
					spam_TF_dict[find] = spam_TF_dict[find] + 1
				except:	
					spam_TF_dict[find] = spam_TF_dict.get(find,1)
					ham_TF_dict[find] = ham_TF_dict.get(find,0)
		else:
			for find in finds:
				if len(find)<ignore: continue
				find = find.lower()
				try:
					ham_TF_dict[find] = ham_TF_dict[find] + 1
				except:	
					spam_TF_dict[find] = spam_TF_dict.get(find,0)
					ham_TF_dict[find] = ham_TF_dict.get(find,1)
		
		word_set = set()
		for find in finds:
			if len(find)<ignore: continue
			find = find.lower()
			if not(find in word_set):
				try:
					IDF_dict[find] = IDF_dict[find] + 1
				except:	
					IDF_dict[find] = IDF_dict.get(find,1)
			word_set.add(find)

	word_df = pd.DataFrame(list(zip(ham_TF_dict.keys(),ham_TF_dict.values(),spam_TF_dict.values(),IDF_dict.values())))
	word_df.columns = ['keyword','ham_TF','spam_TF','IDF']
	word_df['ham_TF'] = word_df['ham_TF'].astype('float')/train_df[train_df['label']=='ham'].shape[0]
	word_df['spam_TF'] = word_df['spam_TF'].astype('float')/train_df[train_df['label']=='spam'].shape[0]
	word_df['IDF'] = np.log10(train_df.shape[0]/word_df['IDF'].astype('float'))
	word_df['ham_TFIDF'] = word_df['ham_TF']*word_df['IDF']
	word_df['spam_TFIDF'] = word_df['spam_TF']*word_df['IDF']
	word_df['diff']=word_df['spam_TFIDF']-word_df['ham_TFIDF']

	selected_spam_key = word_df.sort_values('diff',ascending=False)

	print('>>>Generating Tabu List...\n  Tabu List Size: {}\n  File Name: {}\n  The words shorter than {} are ignored by model\n'.format(tabu_size, path, ignore))
	file = open(path,'w')
	for word in selected_spam_key.head(tabu_size).keyword:
		file.write(word+'\n')
	file.close()

4. Leia a Lista Tabu e converta SMS
-----------------------------------
Como a mensagem é de tamanho variável, não é fácil para a implementação do algoritmo de aprendizado. Portanto, definimos uma Função acima, gerando uma lista de tabu e armazenando-a no arquivo local. E podemos usar esse arquivo para converter um SMS expresso em string em um vetor de comprimento fixo expresso em valor binário.

A ideia é dada assim: Se tivermos uma lista de tabu, poderemos encontrar essas palavras na lista e representá-las por um índice. Assim, uma string pode ser convertida em uma matriz de int. Além disso, poderíamos definir uma matriz preenchida com zeros com o mesmo comprimento da lista de tabu. se este str contiver a palavra na lista tabu, poderíamos atribuir 1 ao elemento correspondente da matriz que representa 'message contains word w'. (dicas: a consulta de `python.dict` é de tempo constante, muito mais rápida que` python.list`)

Ao executar esta etapa, poderíamos converter nossos dados brutos de comprimento de variante em dados numéricos de comprimento fixo.

Estas duas funções são dadas abaixo:

In [ ]:
def read_tabu_list():
	file = open('tabu.txt','r')
	keyword_dict = dict()
	i = 0
	for line in file:
		keyword_dict.update({line.strip():i})
		i+=1
	return keyword_dict

def convert_Content(content, tabu):
	m = len(tabu)
	res = np.int_(np.zeros(m))
	finds = re.findall('[A-Za-z]+', content)
	for find in finds:
		find=find.lower()
		try:
			i = tabu[find]
			res[i]=1
		except:
			continue
	return res

5. Aprendendo, testando e prevendo
-----------------------------------
Depois de gerar nossa lista de tabu e as funções de suporte, agora estamos bem preparados para a parte de aprendizado desse problema. E aqui podemos usar a biblioteca `sklearn.naive_bayes import BernoulliNB`. Isso nos ajudará a treinar esse modelo.

Antes desta parte, vamos revisar nossos dados: nossa entrada de recurso X é uma matriz * m, onde X [i, j] = 1 significa que a amostra #i contém a palavra j na lista de guias e o rótulo supervisionado Y é um * 1 vetor onde Y [i] = 1 representando para um spam e 0 para um presunto.

Vamos preparar os materiais para o algoritmo de aprendizado.

In [ ]:
def learn():
	global tabu, m
	train,_ = readData()
	n = train.shape[0]
	X = np.zeros((n,m)); Y=np.int_(train.label=='spam')
	for i in range(n):
		X[i,:] = convert_Content(train.iloc[i].content, tabu)

	NaiveBayes = BernoulliNB()
	NaiveBayes.fit(X, Y)

	Y_hat = NaiveBayes.predict(X)
	print('>>>Learning...\n  Learning Sample Size: {}\n  Accuracy (Training sample): {:.2f}％\n'.format(n,sum(np.int_(Y_hat==Y))*100./n))
	return NaiveBayes

A Função acima retorna um objeto Naive Bayes Model bem treinado, e poderíamos usá-lo para fazer previsões.

In [ ]:
def test(NaiveBayes):
	global tabu, m
	_,test = readData()
	n = test.shape[0]
	X = np.zeros((n,m)); Y=np.int_(test.label=='spam')
	for i in range(n):
		X[i,:] = convert_Content(test.iloc[i].content, tabu)
	Y_hat = NaiveBayes.predict(X)
	print ('>>>Cross Validation...\n  Testing Sample Size: {}\n  Accuracy (Testing sample): {:.2f}％\n'.format(n,sum(np.int_(Y_hat==Y))*100./n))
	return

def predictSMS(SMS):
	global NaiveBayes, tabu, m
	X = convert_Content(SMS, tabu)
	Y_hat = NaiveBayes.predict(X.reshape(1,-1))
	if int(Y_hat) == 1:
		print ('SPAM: {}'.format(SMS))
	else:
		print ('HAM: {}'.format(SMS))

6. Montagem Geral
-------------------
Depois de definirmos todos os módulos que precisamos neste problema, poderemos integrá-los em uma parte inteira.**

In [ ]:
print('UCI SMS SPAM CLASSIFICATION PROBLEM SET\n  -- implemented by Bernoulli Naive Bayes Model\n')
tabu_file = 'tabu.txt'          # user defined tabu file
tabu_size = 300                 # how many features are used to classify spam
word_len_ignored = 3            # ignore those words shorter than this variable
# build a tabu list based on the training data
generate_tabu_list(tabu_file,tabu_size, word_len_ignored)

tabu = read_tabu_list()
m = len(tabu)
# train the Naive Bayes Model using training data
NaiveBayes=learn()
# Test Model using testing data
test(NaiveBayes)
print('>>>Testing')
# I select two messages from the test data here.
predictSMS('Ya very nice. . .be ready on thursday')
predictSMS('Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call')

Ok, funciona! Uma precisão de 98,28% no conjunto de treinamento e 97,77% é aceitável para mim.

Por favor, não hesite em me perguntar, se você tiver alguma dúvida. E se você gosta deste guia, faça um voto positivo, Muito obrigado.

--Clyde Wang

In [ ]:
train, _ = readData()
n = train.shape[0]
X = np.zeros((n,m))
Y=np.int_(train.label=='spam')
for i in range(n):
    X[i,:] = convert_Content(train.iloc[i].content, tabu)
Y_hat = NaiveBayes.predict(X)
from sklearn.metrics import classification_report
print(classification_report(Y,Y_hat))